In [1]:
import pandas as pd
import numpy as np

# Иерархическая индексация 

In [2]:
df = pd.DataFrame([
    ['name', 'age', 'city'],
    ['Danila', 21, 'Voronezh'],
    ['Kate', 21, 'Voronezh'],
    ['Vlad', 20, 'Moscow'],
])
df

,0,1,2
0,name,age,city
1,Danila,21,Voronezh
2,Kate,21,Voronezh
3,Vlad,20,Moscow


In [14]:
df.stack()

0  0        name
   1         age
   2        city
1  0      Danila
   1          21
   2    Voronezh
2  0        Kate
   1          21
   2    Voronezh
3  0        Vlad
   1          20
   2      Moscow
dtype: object

In [13]:
df.stack()[1][0]

'Danila'

In [13]:
df.stack().loc[1, 0]

'Danila'

### Мультииндекс 

In [20]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [25]:
index = pd.MultiIndex.from_tuples(index)
index.levels

FrozenList([['California', 'New York', 'Texas'], [2000, 2010]])

In [26]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [38]:
pop[:, 2000]  # выбор всех данных, второй индекс которых равен 2010

California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [54]:
pop.index.names = ['state', 'year']  # задать названия для уровней объекта MultiIndex
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [40]:
pop_df = pop.unstack()  # преобразует Мультииндекс в индексированный DataFrame
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [41]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Методы создания мультииндексов

In [44]:
# передать в конструктор 2+ индексируемых массива
df = pd.DataFrame(np.random.rand(4, 2), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=['data1', 'data2'])
df

data1     data2
a 1  0.990183  0.592191
  2  0.191522  0.264572
b 1  0.406660  0.689122
  2  0.025943  0.465262

In [52]:
# конструктор pd.MultiIndex
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
# pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]]).levels
# pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]]).codes  # codes == labels
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### Индексация и срезы по мультииндексу 

In [55]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [57]:
pop['California', 2010]  # обращение к отдельным элементам путем индексации с помощью нескольких термов

37253956

In [68]:
pop.loc['California':'New York']  # частичный срез

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [69]:
pop[pop > 20_000_000]  # булева маска

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [71]:
pop[['California', 'Texas']]  # прихотливая индексация

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Перегруппировка мультииндексов 

In [75]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.059933
      2      0.904148
c     1      0.508511
      2      0.595962
b     1      0.044505
      2      0.921280
dtype: float64

In [81]:
# большинство операций срезов с мультииндексом завершится ошибкойЮ если индекс не отсортирован
data = data.sort_index()
data

char  int
a     1      0.059933
      2      0.904148
b     1      0.044505
      2      0.921280
c     1      0.508511
      2      0.595962
dtype: float64

In [85]:
data['a':'b']

char  int
a     1      0.059933
      2      0.904148
b     1      0.044505
      2      0.921280
dtype: float64

In [90]:
data.unstack(level=0)

char,a,b,c
int,,,
1,0.059933,0.044505,0.508511
2,0.904148,0.921280,0.595962


In [91]:
data.unstack(level=1)

int,1,2
char,,
a,0.059933,0.904148
b,0.044505,0.921280
c,0.508511,0.595962


### Создание и перестройка индексов

In [96]:
pop_flat = pop.reset_index(name='population')  # преобразовать метки индекса в значения столбцов
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [97]:
pop_flat.set_index(['state', 'year'])  # создать мультииндекс из значений столбцов

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Агрегирование по мультииндексам 

In [98]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [109]:
pop.mean(level='year')
# аналоги: 
# pop.mean(axis = 0, level='year')
# pop.groupby(level='year').mean()

C:\Users\Данила\AppData\Local\Temp\ipykernel_11736\1593595266.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  pop.mean(level='year')


year
2000    2.456664e+07
2010    2.725921e+07
dtype: float64